# Análisis GWAS en Cohortes LAGC Usando Regenie
Autores: Rafaella Ormond y Jose Jaime Martinez-Magana <br>

***Descripción:***<br>
En este análisis, se realizó un GWAS utilizando **Regenie** para una cohorte en el LAGC.

**ENTRADA:**
1) Datos de genotipo en formato PLINK pgen/pvar/sample (.pgen, .pvar, .sample)
2) Archivos de fenotipo y covariables correspondientes a la cohorte
3) Covariables para el ajuste en el GWAS

**SALIDA:** Resultados de asociación GWAS de Regenie paso 2 (archivos de regenie con estadísticas resumen)

### ***Requisitos:***

**Regenie** está diseñado para su uso con cohortes que tienen un tamaño de muestra mayor a 300.

### Descargar Plink
Podemos descargar Plink versión 1.9 y versión 2.0 siguiendo los pasos de su sitio web.<br>
Para instalar plink2 [acceda aquí](https://www.cog-genomics.org/plink/2.0/)<br>
Para instalar plink1.9 [acceda aquí](https://www.cog-genomics.org/plink/1.9/) <br>

### Descargar Regenie
Para descargar **Regenie**, siga este enlace: [Acceda aquí](https://github.com/rgcgithub/regenie)

Para más detalles, consulte el plan de análisis: [Acceda aquí](https://docs.google.com/document/d/1RzD5kBlj9rfiomda1G3NfxYDXLdmIUO7VX0cSNj70Kk/edit?usp=sharing)

### Pasos del Análisis:
1) Recodificar pfile a bfile<br>
2) Recodificar pgen a bgen<br>
3) Regenie paso 1 (ajuste del modelo de regresión de genoma completo)<br>
4) Regenie paso 2 (prueba de asociación)

**Descripción:**<br>
Re-codificar archivos en formato PLINK2 a formato bfile, si es necesario.<br>
Ajuste las rutas de entrada y los nombres de archivo según corresponda.

In [ ]:
## Transformar pgen a bfile para el Paso 1
# Este paso convierte el formato .pgen a .bfile para su uso en el paso 1
# Asegúrese de ajustar los nombres de archivo y sufijos según sus archivos PLINK

plink2 \
    --pfile cohort_name \
    --maf 0.01 --mac 100 --geno 0.1 --hwe 1e-10 --mind 0.1 \
    --write-snplist --write-samples --no-id-header --make-bed --snps-only 'just-acgt' --max-alleles 2 \
    --out cohort_name

### 2) Re-codificar pgen a bgen
**Descripción:**<br>
Este paso re-codifica archivos en formato PLINK2 (`.pgen`) a formato (`.bgen`) para su uso en el paso 2.

In [ ]:
## Re-codificar pgen a bgen 
# El archivo pgen arrojó un error en el archivo psam
# Asegúrese de ajustar los nombres de archivo y sufijos según sus archivos PLINK

plink2 --pfile cohort_name --export bgen-1.1 --snps-only 'just-acgt' --max-alleles 2 --out cohort_name --threads 30

# Regenie

Para los pasos 1 y 2 se usarán archivos de fenotipo y covariables, ambos separados por sexo.  

**Los archivos de fenotipo (`inph`)** deben incluir las siguientes columnas:  
`FID`, `IID`, `Pheno1`, `Pheno2`  

Deben existir dos archivos separados:  
- `inph_bt`: para **rasgos binarios**  
- `inph_qt`: para **rasgos cuantitativos**  

Los fenotipos deben seleccionarse y ajustarse en función de los disponibles en su cohorte y según el plan de análisis:  
[Acceder al plan de análisis aquí](https://docs.google.com/document/d/1RzD5kBlj9rfiomda1G3NfxYDXLdmIUO7VX0cSNj70Kk/edit?usp=sharing)  

**El archivo de covariables (`inco`)** debe incluir las siguientes columnas:  
`FID`, `IID`, `age`, y `PC1` a `PC10`  

Las 10 PCs (Componentes Principales) deben generarse usando **PC-AiR**.  
Asegúrese de que los individuos en los archivos de fenotipo y covariables coincidan.

> **Nota:** !!! Advertencia, usaremos SNPs de HapMap para el Paso 1!!!<br>
> Para HapMap seleccionamos 250k SNPs aleatoriamente, por favor descargue este archivo desde GitHub [ENLACE aquí](https://github.com/ormondr/Smoking_GWAS_LAGC/blob/main/English/02GWAS/00Regenie/w_hm3_hg38_random250K.snplist)

### 3) Regenie Paso 1

**Descripción:**<br>
**Paso 1 – Ajuste del modelo de regresión a todo el genoma**<br>
Este paso ajusta un modelo de regresión ridge a lo largo del genoma usando datos de genotipo.  
El objetivo es estimar predicciones individuales del rasgo (leave-one-chromosome-out, LOCO) teniendo en cuenta estructura poblacional y parentesco.  
Entrada: datos de genotipo en formato PLINK y archivos de fenotipo/covariables.  
Salida: predicciones LOCO usadas como covariables en el Paso 2.

> **Nota:** !!! Advertencia, usaremos SNPs de HapMap para el Paso 1!!!<br>
> Para HapMap seleccionamos 250k SNPs aleatoriamente, por favor descargue este archivo desde GitHub [ENLACE aquí]()

> **Nota:** <br>
> Los archivos `"inph"` contienen la información de fenotipo<br>
> Los archivos `"inco"` contienen la información de covariables, `"bt"` para rasgos binarios y `"qt"` para rasgos cuantitativos<br>
> `"bt"` para rasgos binarios y "qt" para rasgos cuantitativos<br>
> **El análisis debe separarse por sexo**

In [ ]:
## Establecer parámetros
# Establecer datos de genotipo
# Ajuste según sus datos
# El "inge" debe ser el prefijo/sufijo de los archivos plink
inge="path/cohort_name"

# Los archivos "inph" contienen la información de fenotipo, "bt" para rasgos binarios y "qt" para rasgos cuantitativos, separados por sexo
# Establecer archivos de fenotipo y covariables para mujeres
female_inph_bt="path/cohort_female_pheno_bt_forregenie.txt"
female_inph_qt="path/cohort_female_pheno_qt_forregenie.txt"
female_inco="path/cohort_female_covar_forregenie.txt"

# Establecer archivos de fenotipo y covariables para hombres
male_inph_bt="path/cohort_male_pheno_bt_forregenie.txt"
male_inph_qt="path/cohort_male_pheno_qt_forregenie.txt"
male_inco="path/cohort_male_covar_forregenie.txt"

# Establecer salida para el paso 1
out_step1_qt_female="path/cohort/out_step1_qt_female"
out_step1_bt_female="path/cohort/out_step1_bt_female"
out_step1_qt_male="path/cohort/out_step1_qt_male"
out_step1_bt_male="path/cohort/out_step1_bt_male"

# Establecer salida para el paso 2
out_step2_qt_female="path/cohort/out_step2_qt_female"
out_step2_bt_female="path/cohort/out_step2_bt_female"
out_step2_qt_male="path/cohort/out_step2_qt_male"
out_step2_bt_male="path/cohort/out_step2_bt_male"

## Agregar ceros al FID si obtiene errores. Puede usar el siguiente código en bash
# awk 'BEGIN {OFS=\
} NR==1 {print \"FID\", $0; next} {print \"0\", $0}' "${female_inph_qt}" > tmpfile && mv tmpfile "${female_inph_qt}"
## !!! Advertencia, usaremos SNPs de HapMap para el Paso 1!!!
# Para HapMap seleccionamos 250k SNPs aleatoriamente, por favor descargue este archivo desde GitHub
hapmap="/vast/palmer/scratch/montalvo-ortiz/jjm262/02lagc_smoking_gwas/11references/w_hm3_hg38_random250K.snplist"

## Ejecutando Paso1
# Ejecutando para mujeres, rasgos cuantitativos
regenie\
    --step 1\
    --bed ${inge}\
    --covarFile ${female_inco}\
    --phenoFile ${female_inph_qt}\
    --bsize 400\
    --qt\
    --extract ${hapmap}\
    --force-step1\
    --out ${out_step1_qt_female}

# Ejecutando para mujeres, rasgos binarios
regenie\
    --step 1\
    --bed ${inge}\
    --covarFile ${female_inco}\
    --phenoFile ${female_inph_bt}\
    --bsize 400\
    --iid-only\
    --bt\
    --extract ${hapmap}\
    --force-step1\
    --out ${out_step1_bt_female}

# Ejecutando para hombres, rasgos cuantitativos
regenie\
    --step 1\
    --bed ${inge}\
    --covarFile ${male_inco}\
    --phenoFile ${male_inph_qt}\
    --bsize 400\
    --iid-only\
    --qt\
    --extract ${hapmap}\
    --force-step1\
    --out ${out_step1_qt_male}

# Ejecutando para hombres, rasgos binarios
regenie\
    --step 1\
    --bed ${inge}\
    --covarFile ${male_inco}\
    --phenoFile ${male_inph_bt}\
    --bsize 400\
    --iid-only\
    --bt\
    --extract ${hapmap}\
    --force-step1\
    --out ${out_step1_bt_male}

### 4) Regenie Step 2

**Description:**
**Step 2 – Association testing:** <br>
This step performs the GWAS using the LOCO predictions from Step 1 as offsets in a linear/firth regression model.  
It tests each variant for association with the trait of interest.  
Input: same phenotype/covariate files, LOCO predictions, and full genotype data.  
Output: GWAS results (effect sizes, p-values, etc.).

> **Note:** <br>
> The `"inph"` files are the phenotype information<br>
> The `"inco"` files are the covariate information, `"bt"` for binary traits and `"qt"` for quantitative traits<br>
> `"bt"` for binary traits and "qt" for quantitative traits<br>
> **The analysis needs to be separated by sex**

In [ ]:
## Set parameters
# Set genotype data
# Adjust acording to your data
# The "inge" needs to be the plink sufix files
inge="path/cohort_name"

# The "inph" files are the phenotype information, "bt" for binary traits and "qt" for quantitative traits, separated by sex
# Set phenotype and covariates files for females
female_inph_bt="path/cohort_female_pheno_bt_forregenie.txt"
female_inph_qt="path/cohort_female_pheno_qt_forregenie.txt"
female_inco="path/cohort_female_covar_forregenie.txt"

# Set phenotype and covariates files for males
male_inph_bt="path/cohort_male_pheno_bt_forregenie.txt"
male_inph_qt="path/cohort_male_pheno_qt_forregenie.txt"
male_inco="path/cohort_male_covar_forregenie.txt"

# Set output for step 1
out_step1_qt_female="path/cohort/out_step1_qt_female"
out_step1_bt_female="path/cohort/out_step1_bt_female"
out_step1_qt_male="path/cohort/out_step1_qt_male"
out_step1_bt_male="path/cohort/out_step1_bt_male"

# Set output for step 2
out_step2_qt_female="path/cohort/out_step2_qt_female"
out_step2_bt_female="path/cohort/out_step2_bt_female"
out_step2_qt_male="path/cohort/out_step2_qt_male"
out_step2_bt_male="path/cohort/out_step2_bt_male"

# Running for females for quantitative traits
regenie \
  --step 2 \
  --bgen ${inge}.bgen \
  --ref-first \
  --sample ${inge}.sample \
  --phenoFile ${female_inph_qt} \
  --covarFile ${female_inco} \
  --qt \
  --pred ${out_step1_qt_female}_pred.list \
  --bsize 400 \
  --out ${out_step2_qt_female}

# Running for females for binary traits
regenie \
  --step 2 \
  --bgen ${inge}.bgen \
  --ref-first \
  --sample ${inge}.sample \
  --phenoFile ${female_inph_bt} \
  --covarFile ${female_inco} \
  --bt \
  --firth --approx --pThresh 0.01 \
  --pred ${out_step1_bt_female}_pred.list \
  --bsize 400 \
  --out ${out_step2_bt_female}

# Running for males for quantitative traits
regenie \
  --step 2 \
  --bgen ${inge}.bgen \
  --ref-first \
  --sample ${inge}.sample \
  --phenoFile ${male_inph_qt} \
  --covarFile ${male_inco} \
  --qt \
  --pred ${out_step1_qt_male}_pred.list \
  --bsize 400 \
  --out ${out_step2_qt_male}

# Running for males for binary traits
regenie \
  --step 2 \
  --bgen ${inge}.bgen \
  --ref-first \
  --sample ${inge}.sample \
  --phenoFile ${male_inph_bt} \
  --covarFile ${male_inco} \
  --bt \
  --firth --approx --pThresh 0.01 \
  --pred ${out_step1_bt_male}_pred.list \
  --bsize 400 \
  --out ${out_step2_bt_male}